In [52]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# load dataset
data_url = "https://raw.githubusercontent.com/ajisakarsyi/ai-project/refs/heads/main/ai-dataset/Data_Tanaman_Padi_Sumatera_version_1.csv"
data = pd.read_csv(data_url)
print(data.head())

data = data.dropna()

  Provinsi  Tahun   Produksi  Luas Panen  Curah hujan  Kelembapan  \
0     Aceh   1993  1329536.0    323589.0       1627.0       82.00   
1     Aceh   1994  1299699.0    329041.0       1521.0       82.12   
2     Aceh   1995  1382905.0    339253.0       1476.0       82.72   
3     Aceh   1996  1419128.0    348223.0       1557.0       83.00   
4     Aceh   1997  1368074.0    337561.0       1339.0       82.46   

   Suhu rata-rata  
0           26.06  
1           26.92  
2           26.27  
3           26.08  
4           26.31  


In [ ]:
# membuat cluster
kmeans = KMeans(n_clusters=4, random_state=42)
data['Produksi_Cluster'] = kmeans.fit_predict(data[['Produksi']])

# label cluster
cluster_centers = kmeans.cluster_centers_.flatten()
sorted_centroids = np.sort(cluster_centers)
cluster_ranges = {
    "Low Production": sorted_centroids[0],
    "Moderate Production": sorted_centroids[1],
    "High Production": sorted_centroids[2],
    "Very High Production": sorted_centroids[3]
}

# map cluster
def get_cluster_label(value):
    if value <= sorted_centroids[0]:
        return "Low Production"
    elif value <= sorted_centroids[1]:
        return "Moderate Production"
    elif value <= sorted_centroids[2]:
        return "High Production"
    else:
        return "Very High Production"

# split data x input dan y target
X = data[['Luas Panen', 'Curah hujan', 'Kelembapan', 'Suhu rata-rata']]
y = data['Produksi']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# normalisasi
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [53]:
# neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2), 
    Dense(64, activation='relu'),
    Dropout(0.2), 
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])


c:\Users\Muhammad Ajisaka\Downloads\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [54]:
# train model
epochs = 200
batch_size = 16
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 4079051866112.0000 - mae: 1657553.7500 - val_loss: 3087897722880.0000 - val_mae: 1484509.6250
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4670995038208.0000 - mae: 1792382.1250 - val_loss: 3087896936448.0000 - val_mae: 1484509.5000
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4768742768640.0000 - mae: 1762885.2500 - val_loss: 3087895363584.0000 - val_mae: 1484509.0000
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4813786447872.0000 - mae: 1802923.5000 - val_loss: 3087891431424.0000 - val_mae: 1484507.7500
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4288672956416.0000 - mae: 1694811.6250 - val_loss: 3087881732096.0000 - val_mae: 1484504.3750
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4395217190912.0000 - mae: 1719506.0000 - val_loss: 3087857090560.0000 - val_mae: 1484496.7500
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4469556772864.0000 - m

In [55]:
# fungsi prediksi
def predict_production(luas_panen, curah_hujan, kelembapan, suhu_rata_rata):
    weather_features = np.array([[luas_panen, curah_hujan, kelembapan, suhu_rata_rata]])
    weather_features = scaler.transform(weather_features)
    prediction = model.predict(weather_features)[0][0]

    cluster_category = get_cluster_label(prediction)

    return prediction, cluster_category

In [ ]:
# contoh input
luas_panen = 300000
curah_hujan = 1600
kelembapan = 90
suhu_rata_rata = 20
predicted_production, production_cluster = predict_production(luas_panen, curah_hujan, kelembapan, suhu_rata_rata)
print(f"Predicted Production: {predicted_production}")
print(f"Production Cluster: {production_cluster}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Predicted Production: 1997715.625
Production Cluster: High Production


c:\Users\Muhammad Ajisaka\Downloads\myenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
# save model
model.save('ai-model.h5')
